In [8]:
import sys
sys.path.append('../')
from model.op import number_aircrafts_lp
import numpy as np

In [9]:
tau_list = [[[0, 5.92], [5.85, 0]],
            [[0, 9.82], [10.59, 0]],
            [[0, 13.40], [13.59, 0]],
            [[0, 19.35], [19.39, 0]],
            [[0, 23.47], [23.67, 0]],
            [[0, 28.19], [28.20, 0]],
            [[0, 32.17], [32.39, 0]],
            [[0, 36.50], [36.75, 0]],
            [[0, 41.40], [41.37, 0]]]

kappa_list = [[[0, 12.35], [11.91, 0]],
              [[0, 16.04], [17.11, 0]],
              [[0, 18.88], [19.06, 0]],
              [[0, 26.52], [26.71, 0]],
              [[0, 30.35], [31.09, 0]],
              [[0, 36.31], [36.40, 0]],
              [[0, 39.58], [40.16, 0]],
              [[0, 44.05], [44.72, 0]],
              [[0, 50.86], [50.71, 0]]]
# gamma = [3.237872699, 3.729470167, 4.404786588, 5.379957014, 
#          6.913363091, 9.685271742, 16.30528373, 71.41103553]
gamma = [1.567183013,1.670689686,1.79349788,1.935972287,2.103057098,
         2.30172949,2.541890384,2.83806663,3.212473781,3.70088931,
         4.364896382,5.32037536,6.814736187,9.490547548,15.74119426,55.66984127]

In [10]:
for i in range(9):
    tau = tau_list[i]
    print(tau)

    kappa = kappa_list[i]
    kappa = np.array(kappa)
    kappa = kappa / 1.6
    print(kappa)
    number_aircrafts_lp(tau=tau, kappa=kappa, gamma=gamma, schedule='schedule', schedule_time_step=288, output_path='output_' + str((i+1)*12) + ' miles')

[[0, 5.92], [5.85, 0]]
[[0.      7.71875]
 [7.44375 0.     ]]
Set parameter MIPGap to value 0.05
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-10910 CPU @ 3.60GHz
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 11362 rows, 98940 columns and 205478 nonzeros
Model fingerprint: 0x69c44e35
Variable types: 0 continuous, 98940 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 1033 rows and 3188 columns
Presolve time: 0.20s
Presolved: 10329 rows, 95752 columns, 199552 nonzeros
Variable types: 0 continuous, 95752 integer (0 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
 

In [ ]:
for i in range(9):
    tau = tau_list[i]
    print(tau)

    kappa = kappa_list[i]
    kappa = np.array(kappa)
    kappa = kappa / 1.6
    print(kappa)
    number_aircrafts_lp(tau=tau, kappa=kappa, gamma=gamma, schedule='schedule_10min', schedule_time_step=288, output_path='output_' + str((i+1)*12) + ' miles_10min')

[[0, 5.92], [5.85, 0]]
[[0.      7.71875]
 [7.44375 0.     ]]
Set parameter MIPGap to value 0.05
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-10910 CPU @ 3.60GHz
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 11362 rows, 98940 columns and 205478 nonzeros
Model fingerprint: 0xe2dc68fa
Variable types: 0 continuous, 98940 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 1046 rows and 3188 columns
Presolve time: 0.20s
Presolved: 10316 rows, 95752 columns, 199344 nonzeros
Variable types: 0 continuous, 95752 integer (0 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
 